In [2]:
import pandas as pd
import numpy as np
import pandas_ta as ta
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error 
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

In [40]:

# Step 1: Data cleaning
data = yf.download(tickers='BTC-usd', period='max', interval='1d')
data.dropna(inplace=True)
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800
2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200
2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700
2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600
2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100
...,...,...,...,...,...,...
2023-01-10,17192.949219,17484.720703,17162.990234,17446.292969,17446.292969,15808338949
2023-01-11,17446.359375,17934.896484,17337.994141,17934.896484,17934.896484,18372283782
2023-01-12,18117.593750,19030.087891,17995.203125,18869.587891,18869.587891,34971338710


In [41]:
# Step 2: Feature engineering
data['RSI'] = ta.rsi(data.Close, length=15)
data['EMA_short'] = ta.ema(data.Close, length=20)
data['EMA_long'] = ta.ema(data.Close, length=100)
data.dropna(inplace=True)
data.reset_index(inplace = True)
data.drop([ 'Close'], axis=1, inplace=True)
data

,Date,Open,High,Low,Adj Close,Volume,RSI,EMA_short,EMA_long
0,2014-12-25,322.286011,322.670013,316.958008,319.007996,9883640,38.229085,337.460187,366.468409
1,2014-12-26,319.152008,331.424011,316.627014,327.924011,16410500,43.088077,336.551980,365.705154
2,2014-12-27,327.583008,328.911011,312.630005,315.863007,15185200,38.678402,334.581601,364.718181
3,2014-12-28,316.160004,320.028015,311.078003,317.239014,11676600,39.436044,332.929926,363.777999
4,2014-12-29,317.700989,320.266998,312.307007,312.670013,12302500,37.775595,331.000411,362.765960
...,...,...,...,...,...,...,...,...,...
2938,2023-01-10,17192.949219,17484.720703,17162.990234,17446.292969,15808338949,63.070667,16929.558966,18043.148390
2939,2023-01-11,17446.359375,17934.896484,17337.994141,17934.896484,18372283782,70.509668,17025.305396,18041.004788
2940,2023-01-12,18117.593750,19030.087891,17995.203125,18869.587891,34971338710,79.127422,17200.951348,18057.412374
2941,2023-01-13,18868.906250,19964.322266,18753.164062,19909.574219,29225029694,84.520107,17458.915431,18094.088846


In [50]:
# Step 3: Data normalization (cont.)
#scaler = MinMaxScaler()
#data[['Open', 'High', 'Low', 'Volume', 'Close', 'RSI', 'EMA_short', 'EMA_long']] = scaler.fit_transform(data[['Open', 'High', 'Low', 'Volume', 'Close', 'RSI', 'EMA_short', 'EMA_long']])
#data
scaler = MinMaxScaler()
data_close = data[['Adj Close']]
data_close = scaler.fit_transform(data_close)
data_close

array([[0.00209093],
       [0.00222324],
       [0.00204426],
       ...,
       [0.27736813],
       [0.29280078],
       [0.30205023]])

In [51]:
# Step 4: Splitting the data
X = data[['Open', 'High', 'Low', 'Volume', 'RSI', 'EMA_short', 'EMA_long']]
X = scaler.fit_transform(X)
y = data_close
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
print('X', X)


X [[0.00215798 0.00161771 0.00219675 ... 0.33194122 0.0017274  0.00227644]
 [0.00211146 0.00174536 0.00219175 ... 0.39037668 0.00171298 0.00226249]
 [0.0022366  0.00170871 0.00213138 ... 0.33734483 0.00168169 0.00224446]
 ...
 [0.26628976 0.2744085  0.26919717 ... 0.82379484 0.26948845 0.32549977]
 [0.27744132 0.28803146 0.2806449  ... 0.88864861 0.27358445 0.32616987]
 [0.29286181 0.30365316 0.29808418 ... 0.9151398  0.27823292 0.32705221]]


In [52]:
# Step 5: Reshaping the data
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
print("x", X.shape)
print('y',y.shape)
print('X_train', X_train.shape)
print('y_train', y_train.shape)
print('X_test', X_test.shape)
print('Y_test', y_test.shape) 
print('X_train.shape[1]', X_train.shape[0])
print('X_train.shape[2]', X_train.shape[2])

x (2943, 7)
y (2943, 1)
X_train (2354, 1, 7)
y_train (2354, 1)
X_test (589, 1, 7)
Y_test (589, 1)
X_train.shape[1] 2354
X_train.shape[2] 7


In [64]:
# Step 6: Building the LSTM model (cont.)

model = Sequential()
model.add(LSTM(100, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True , activation= 'linear'))
model.add(Dropout(0.2))
model.add(LSTM(100, activation= 'linear'))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

# Step 7: Training and evaluating the model
history = model.fit(X_train, y_train, epochs=250, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/250
74/74 [==============================] - 26s 22ms/step - loss: 0.0120 - val_loss: 0.0015
Epoch 2/250
74/74 [==============================] - 1s 10ms/step - loss: 7.0841e-04 - val_loss: 3.8538e-04
Epoch 3/250
74/74 [==============================] - 1s 10ms/step - loss: 5.9435e-04 - val_loss: 2.9590e-04
Epoch 4/250
74/74 [==============================] - 1s 10ms/step - loss: 6.0148e-04 - val_loss: 0.0011
Epoch 5/250
74/74 [==============================] - 1s 9ms/step - loss: 3.9224e-04 - val_loss: 2.8960e-04
Epoch 6/250
74/74 [==============================] - 1s 11ms/step - loss: 4.5860e-04 - val_loss: 3.6355e-04
Epoch 7/250
74/74 [==============================] - 1s 10ms/step - loss: 3.8097e-04 - val_loss: 2.2935e-04
Epoch 8/250
74/74 [==============================] - 1s 12ms/step - loss: 3.0544e-04 - val_loss: 1.9262e-04
Epoch 9/250
74/74 [==============================] - 1s 12ms/step - loss: 3.0407e-04 - val_loss: 2.7903e-04
Epoch 10/250
74/74 [====================

In [65]:

# Step 8: Tuning the Hyperparameters
# You can use techniques such as grid search or random search to find the best values for the model's hyperparameters

# Step 9: Testing the model
y_pred = model.predict(X_test)
y_pred = y_pred.reshape(-1,1)
y_pred = np.nan_to_num(y_pred)
#y_pred = scaler.inverse_transform(y_pred)
#y_pred = scaler.inverse_transform(y_pred[:,[-1]])
#y_test = scaler.inverse_transform(y_test.reshape(-1, 1))
y_test = np.nan_to_num(y_test)


#evaluat
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = 100 * np.mean(np.abs((y_test - y_pred) / y_test))
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("MAPE: {:.2f}%".format(mape))
print("RMSE: {:.2f}".format(rmse))
print("MSE: {:.2f}".format(mse))
print("MAE:{:.2f}".format(mae))

19/19 [==============================] - 1s 4ms/step
MAPE: 1.51%
RMSE: 0.01
MSE: 0.00
MAE:0.01
